In [71]:
import pandas as pd

In [72]:
demographics = pd.read_csv('Demographics-new.csv')

In [73]:
demographics.fillna(0, inplace=True) 

In [74]:
# First, define AKI diagnoses from non-AKI diagnoses
aki_pre = demographics[demographics['OLD_FLAG']==0]
aki_pos = aki_pre[aki_pre['AKI_DIAGNOSIS_FLAG']== 1]
aki_neg = aki_pre[aki_pre['AKI_DIAGNOSIS_FLAG']==0]

# Clean data sets
del aki_neg['CAUSE']
del aki_pos['CAUSE']

del aki_neg['AKI_DIAGNOSIS_FLAG']
del aki_pos['AKI_DIAGNOSIS_FLAG']

del aki_neg['OLD_FLAG']
del aki_pos['OLD_FLAG']

del aki_neg['OUTSIDE_DEATH_FLAG']
del aki_pos['OUTSIDE_DEATH_FLAG']

del aki_neg['SUBJECT_ID']
del aki_pos['SUBJECT_ID']

del aki_neg['DOB']
del aki_pos['DOB']

del aki_neg['DOD']
del aki_pos['DOD']

In [75]:
aki_pos['DOA']
del aki_pos['DOA']
del aki_neg['DOA']

del aki_pos['delta']

del aki_pos['Unnamed: 0']

In [76]:
len(aki_pos)

2159

In [77]:
aki_pos.head()

,GENDER,ADMIT_AGE,ETHNICITY,MARITAL_STATUS,LANGUAGE,RELIGION,INSURANCE,ADMISSION_LOCATION,#ADMISSIONS,DEATH_FLAG,...,VALUENUM-8-7-6-5-4-3-2,ITEMID-3,VALUENUM-8-7-6-5-4-3,ITEMID,VALUENUM-8-7-6-5-4,VALUENUM-8-7-6-5,VALUENUM-8-7-6,VALUENUM-8-7,VALUENUM-8,VALUENUM
10,M,80.564384,OTHER,SINGLE,UKNOWN,OTHER,Medicare,EMERGENCY ROOM ADMIT,1,1,...,0.0,0.0,0.0,0.0,0.0,13.0,25.5,9.1,3.4,144.0
31,M,79.989041,WHITE,MARRIED,UKNOWN,JEWISH,Medicare,EMERGENCY ROOM ADMIT,1,1,...,0.0,0.0,0.0,0.0,0.0,18.2,30.4,10.7,15.3,128.0
32,M,76.687671,WHITE,SINGLE,UKNOWN,EPISCOPALIAN,Medicare,CLINIC REFERRAL/PREMATURE,1,1,...,0.0,0.0,0.0,0.0,0.0,20.2,26.7,8.7,5.5,135.0
34,M,50.520548,WHITE,MARRIED,ENGL,PROTESTANT QUAKER,Private,CLINIC REFERRAL/PREMATURE,2,1,...,0.0,0.0,0.0,0.0,0.0,13.8,20.4,6.5,25.5,230.0
52,F,66.019178,HISPANIC OR LATINO,SEPARATED,UKNOWN,CATHOLIC,Medicare,EMERGENCY ROOM ADMIT,1,1,...,0.0,0.0,0.0,0.0,0.0,14.3,30.2,9.7,8.8,145.0


In [78]:
# Create outcome data sets
aki_sepsis = pd.Series(aki_pos['KIDNEY_FAILURE_FLAG'])
aki_death = pd.Series(aki_pos['DEATH_FLAG'])

In [79]:
del aki_pos['DEATH_FLAG']
del aki_pos['ITEMID']
del aki_pos['ITEMID-3']
del aki_pos['ITEMID-3-2']
del aki_pos['ITEMID-3-2-1']



In [80]:
# Get dummies
aki_pos = pd.get_dummies(aki_pos, columns=['GENDER','ETHNICITY','MARITAL_STATUS', 'LANGUAGE', 'RELIGION', 'INSURANCE', 'ADMISSION_LOCATION'])

In [81]:
# Check outcome numbers
print(aki_sepsis.value_counts())
print(aki_death.value_counts())

0    1998
1     161
Name: KIDNEY_FAILURE_FLAG, dtype: int64
1    1208
0     951
Name: DEATH_FLAG, dtype: int64


In [82]:
aki_pos.head()

,ADMIT_AGE,#ADMISSIONS,CANCER_FLAG,KIDNEY_FAILURE_FLAG,HYPO_FLAG,SHOCK_FLAG,ANEMIA_FLAG,HEMO_FLAG,ACID_FLAG,LIVER_FLAG,...,INSURANCE_Medicaid,INSURANCE_Medicare,INSURANCE_Private,INSURANCE_Self Pay,ADMISSION_LOCATION_CLINIC REFERRAL/PREMATURE,ADMISSION_LOCATION_EMERGENCY ROOM ADMIT,ADMISSION_LOCATION_PHYS REFERRAL/NORMAL DELI,ADMISSION_LOCATION_TRANSFER FROM HOSP/EXTRAM,ADMISSION_LOCATION_TRANSFER FROM OTHER HEALT,ADMISSION_LOCATION_TRANSFER FROM SKILLED NUR
10,80.564384,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
31,79.989041,1,1,1,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0
32,76.687671,1,1,1,0,0,0,0,0,0,...,0,1,0,0,1,0,0,0,0,0
34,50.520548,2,1,0,0,0,0,0,0,0,...,0,0,1,0,1,0,0,0,0,0
52,66.019178,1,1,0,0,0,0,0,0,0,...,0,1,0,0,0,1,0,0,0,0


In [83]:
# Normalize data
from sklearn import preprocessing
aki_pos = preprocessing.minmax_scale(aki_pos)


In [90]:
# We got a 0.71 10-fold cv accuracy score for AKI patients within 1 year mortality
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score
base_model_XG = GradientBoostingClassifier()
scores = cross_val_score(base_model_XG, aki_pos, aki_death, cv=15)
scores.mean()

0.7137173012173013